In [10]:
import numpy as np
import pandas as pd
import os
import math
from datetime import datetime, timedelta
from sklearn import linear_model

In [100]:
data_directory = "../Data/Study/"
hololenses = ['A/', 'B/']
intermediate = ''
subdirectory = "shifted/"

In [19]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['Movement'] != "transition") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    #result = result[result['Movement'] != "transition"]
    #result = result[result['EyeTrackingDataValid'] == 'TRUE']
    #result = result[result['GazeInputSource'] == 'Eyes']
    result.reset_index(inplace=True, drop=True)
    return result

In [16]:
# def create_calibration_df(taskName):
#     frames = []
#     for file in fileList:
#         if taskName in file:
#             df = pd.read_csv(data_directory + subdirectory + file)
#             df = clean_df(df)
#             frames.append(df)
#     result = pd.concat(frames)
#     return result

In [90]:
def create_calibration_df(taskName, fileList, participant, hololens):
    frames = []
    for file in fileList:
        if taskName in file:
            filePath = data_directory + hololens + participant + '/' + intermediate + subdirectory + file
            df = pd.read_csv(filePath)
            df = clean_df(df)
            frames.append(df)
    if len(frames) == 0:
        print("error: task not found")
        return None
    else:
        result = pd.concat(frames)
        return result

In [14]:
def calculate_regression_coefficients(df):
    x1 = df[['GazeAngleX','GazeAngleY']]
    y1 = df['TargetAngleX']
    x2 = df[['GazeAngleX','GazeAngleY']]
    y2 = df['TargetAngleY']
    x_regr = linear_model.LinearRegression()
    x_regr.fit(x1.to_numpy(), y1.to_numpy())
    y_regr = linear_model.LinearRegression()
    y_regr.fit(x2.to_numpy(), y2.to_numpy())
    return [x_regr, y_regr]

In [88]:
def calibrate_files(taskname):
    output_subdirectory = intermediate + "recalibrated_" + taskname
    for hololens in hololenses:
        participantList = os.listdir(data_directory + hololens)
        for participant in participantList:
            print(participant)
            if participant == '.DS_Store':
                print("skipping calibration")
                continue
            if os.path.exists(data_directory + hololens + participant + '/' + output_subdirectory):
                print("already calibrated")
                continue
            fileList = os.listdir(data_directory + hololens + participant + '/' + intermediate + subdirectory)
            calibration_df = create_calibration_df(taskname, fileList, participant, hololens)
            [x_coeff, y_coeff] = calculate_regression_coefficients(calibration_df)
            for file in fileList:
                filePath = data_directory + hololens + participant + '/' + intermediate + subdirectory + file
                if '.csv' in file:
                    df = pd.read_csv(filePath)
                    df = clean_df(df)
                    df['CalibratedGazeAngleX'] = df.apply(lambda row: x_coeff.predict([[row['GazeAngleX'],row['GazeAngleY']]])[0], axis = 1)
                    df['CalibratedGazeAngleY'] = df.apply(lambda row: y_coeff.predict([[row['GazeAngleX'],row['GazeAngleY']]])[0], axis = 1)
                    recalibrated_df_output_directory = data_directory + hololens + participant + '/' + output_subdirectory
                    recalibrated_df_output_path = data_directory + hololens + participant + '/' + output_subdirectory + '/' + file
                    if not os.path.exists(recalibrated_df_output_directory):
                        os.mkdir(recalibrated_df_output_directory)
                    #filePrefix = file.split('.')[0]
                    #recalibrated_df_output_path = data_directory + output_subdirectory + '/' + filePrefix + '_recalibrated_' + taskname + '.csv'
                    df.to_csv(recalibrated_df_output_path, index=False)

In [21]:
# def calibrate_files(taskname, output_subdirectory):
#     calibration_df = create_calibration_df(taskname)
#     [x_coeff, y_coeff] = calculate_regression_coefficients(calibration_df)
#     for file in fileList:
#         if '.csv' in file:
#             df = pd.read_csv(data_directory + subdirectory + file)
#             df = clean_df(df)
#             df['CalibratedGazeAngleX'] = df.apply(lambda row: x_coeff.predict([[row['GazeAngleX'],row['GazeAngleY']]])[0], axis = 1)
#             df['CalibratedGazeAngleY'] = df.apply(lambda row: y_coeff.predict([[row['GazeAngleX'],row['GazeAngleY']]])[0], axis = 1)
#             if not os.path.exists(os.path.join(data_directory, output_subdirectory)):
#                 os.mkdir(os.path.join(data_directory, output_subdirectory))
#             filePrefix = file.split('.')[0]
#             #recalibrated_df_output_path = data_directory + output_subdirectory + '/' + filePrefix + '_recalibrated_' + taskname + '.csv'
#             recalibrated_df_output_path = data_directory + output_subdirectory + '/' + file
#             df.to_csv(recalibrated_df_output_path)

In [101]:
calibrate_files('calibration')

03
already calibrated
04
already calibrated
32
already calibrated
34
already calibrated
05
already calibrated
02
already calibrated
.DS_Store
skipping calibration
18
already calibrated
27
already calibrated
29
already calibrated
10
already calibrated
19
already calibrated
43
already calibrated
07
already calibrated
36
already calibrated
09
already calibrated
37
already calibrated
08
already calibrated
01
already calibrated
06
already calibrated
39
already calibrated
46
already calibrated
41
already calibrated
49
already calibrated
47
already calibrated
25
already calibrated
50
already calibrated
35
already calibrated
51
33
already calibrated
.DS_Store
skipping calibration
20
already calibrated
11
already calibrated
16
already calibrated
42
already calibrated
45
already calibrated
28
already calibrated
17
already calibrated
26
already calibrated
21
already calibrated
44
already calibrated
38
already calibrated
31
already calibrated
30
already calibrated
48
already calibrated
24
already 

In [102]:
tasknames = ['calibration', 'ssHeadConstrained', 'wsBodyConstrained', 'ssWalking', 'wsWalking', 'hallway']
for taskname in tasknames:
    print(taskname)
    calibrate_files(taskname)

calibration
03
already calibrated
04
already calibrated
32
already calibrated
34
already calibrated
05
already calibrated
02
already calibrated
.DS_Store
skipping calibration
18
already calibrated
27
already calibrated
29
already calibrated
10
already calibrated
19
already calibrated
43
already calibrated
07
already calibrated
36
already calibrated
09
already calibrated
37
already calibrated
08
already calibrated
01
already calibrated
06
already calibrated
39
already calibrated
46
already calibrated
41
already calibrated
49
already calibrated
47
already calibrated
25
already calibrated
50
already calibrated
35
already calibrated
51
already calibrated
33
already calibrated
.DS_Store
skipping calibration
20
already calibrated
11
already calibrated
16
already calibrated
42
already calibrated
45
already calibrated
28
already calibrated
17
already calibrated
26
already calibrated
21
already calibrated
44
already calibrated
38
already calibrated
31
already calibrated
30
already calibrated
48